In [1]:
import sys
import os

PROJECT_DIR = '/home/evgenshuben/Desktop/gitReps/YandexCup/'
sys.path.append(PROJECT_DIR)
os.chdir(PROJECT_DIR)

In [2]:
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra import initialize, compose, initialize_config_dir
from hydra.utils import instantiate

import torch.nn as nn
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
from functools import reduce

In [3]:
%load_ext autoreload
%autoreload 2

from scr.train import Trainer
from scr.utils import (reduce_func, save_test_predictions)

# info

Тут попробуем сделать стекинг нескольких моделей предобученных

Пробую взять вектора от моделей на тесте и усреднить их просто

Написал функцию для сохранения тестовых эмбеддингов

# Load config

In [4]:
# config_dir = '/home/evgenshuben/Desktop/gitReps/YandexCup/configs'
# config_name = 'config'


# with initialize_config_dir(config_dir=config_dir, version_base=None):
#     cfg = compose(config_name=config_name)
#     # cfg['dataset']['train']['train']['dataset_path']

In [5]:
# # Путь к конфигурационному файлу, сохраненному Hydra
# config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-2/hydra/.hydra/config.yaml"



# cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
# cfg.model

{'_target_': 'scr.models.cnn.CNN1d', 'emb_size': 512, 'num_classes': '${num_classes}', 'input_norm': True}

In [5]:
# trainer = Trainer(cfg)

### embeddings calculate

Насчитаем эмбеддинги на каждой сильной модели

###### 6

In [6]:
# # Путь к конфигурационному файлу, сохраненному Hydra
# config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-6/hydra/.hydra/config.yaml"
# cfg = OmegaConf.load(config_path)
# # cfg.enviroment.device = 'cpu'
# cfg.model

In [7]:
# trainer = Trainer(cfg)

In [8]:
# %%time
# trainer.test_embeddings()

###### 4

In [9]:
# # Путь к конфигурационному файлу, сохраненному Hydra
# config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-4/hydra/.hydra/config.yaml"
# cfg = OmegaConf.load(config_path)
# # cfg.enviroment.device = 'cpu'
# cfg.model

In [10]:
# trainer = Trainer(cfg)

In [11]:
# %%time
# trainer.test_embeddings()

###### 14

In [12]:
# # Путь к конфигурационному файлу, сохраненному Hydra
# config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-14/hydra/.hydra/config.yaml"
# cfg = OmegaConf.load(config_path)
# # cfg.enviroment.device = 'cpu'
# cfg.model

In [13]:
# trainer = Trainer(cfg)

In [14]:
# %%time
# trainer.test_embeddings()

###### 15

In [31]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-15/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
print(cfg.model.backbone_name)
print(cfg.pipeline.model_ckpt)

hgnetv2_b4.ssld_stage2_ft_in1k
/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-15/data/model/best-model-epoch=1-max_secs=50.pt


In [32]:
trainer = Trainer(cfg)

/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

In [33]:
%%time
trainer.test_embeddings()

100%|██████████| 55170/55170 [05:07<00:00, 179.37it/s]


CPU times: user 5min 7s, sys: 24.5 s, total: 5min 31s
Wall time: 5min 8s


###### 17

In [10]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-17/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
print(cfg.model.backbone_name)
print(cfg.pipeline.model_ckpt)

edgenext_base.in21k_ft_in1k
/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-17/data/model/best-model-epoch=1-max_secs=50.pt


In [5]:
trainer = Trainer(cfg)

/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

In [6]:
%%time
trainer.test_embeddings()

100%|██████████| 55170/55170 [05:20<00:00, 172.15it/s]


CPU times: user 5min 20s, sys: 24.3 s, total: 5min 45s
Wall time: 5min 20s


###### 18

In [4]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-18/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
print(cfg.model.backbone_name)
print(cfg.pipeline.model_ckpt)

tf_efficientnet_b4.ns_jft_in1k
/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-18/data/model/best-model-epoch=0-max_secs=50.pt


In [5]:
trainer = Trainer(cfg)

/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

In [6]:
%%time
trainer.test_embeddings()

100%|██████████| 55170/55170 [08:55<00:00, 103.05it/s]


CPU times: user 8min 55s, sys: 26.9 s, total: 9min 22s
Wall time: 8min 55s


###### 21

In [9]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-21/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
print(cfg.model.backbone_name)
print(cfg.pipeline.model_ckpt)

rexnetr_300.sw_in12k_ft_in1k
/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-21/data/model/best-model-epoch=0-max_secs=50.pt


In [10]:
trainer = Trainer(cfg)

/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

In [11]:
%%time
trainer.test_embeddings()

100%|██████████| 55170/55170 [05:45<00:00, 159.82it/s]


CPU times: user 5min 34s, sys: 26 s, total: 5min 59s
Wall time: 5min 45s


###### 22

In [12]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-22/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
print(cfg.model.backbone_name)
print(cfg.pipeline.model_ckpt)

efficientnet_b5.sw_in12k_ft_in1k
/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-22/data/model/best-model-epoch=0-max_secs=50.pt


In [13]:
trainer = Trainer(cfg)

/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

In [14]:
%%time
trainer.test_embeddings()

100%|██████████| 55170/55170 [10:08<00:00, 90.69it/s]


CPU times: user 10min 8s, sys: 27.4 s, total: 10min 35s
Wall time: 10min 8s


In [85]:
trainer.train_dataloader

###### 26

In [ ]:
# # Путь к конфигурационному файлу, сохраненному Hydra
# config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-26/hydra/.hydra/config.yaml"
# cfg = OmegaConf.load(config_path)
# # cfg.enviroment.device = 'cpu'
# print(cfg.model.backbone_name)
# print(cfg.pipeline.model_ckpt)

In [ ]:
# trainer = Trainer(cfg)

In [ ]:
# %%time
# trainer.test_embeddings()

###### 27

In [ ]:
# # Путь к конфигурационному файлу, сохраненному Hydra
# config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-27/hydra/.hydra/config.yaml"
# cfg = OmegaConf.load(config_path)
# # cfg.enviroment.device = 'cpu'
# print(cfg.model.backbone_name)
# print(cfg.pipeline.model_ckpt)

In [ ]:
# trainer = Trainer(cfg)

In [ ]:
# %%time
# trainer.test_embeddings()

###### 28

In [ ]:
# # Путь к конфигурационному файлу, сохраненному Hydra
# config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-28/hydra/.hydra/config.yaml"
# cfg = OmegaConf.load(config_path)
# # cfg.enviroment.device = 'cpu'
# print(cfg.model.backbone_name)
# print(cfg.pipeline.model_ckpt)

In [ ]:
# trainer = Trainer(cfg)

In [ ]:
# %%time
# trainer.test_embeddings()

### Embeddings pooling

Тут попробую усреднить эмбеды от предобученных моделей
с нормировкой эмбедов

Нужно указать файл с тестов от моделей

In [15]:
# emb_6_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-6/data/test_embeddings.pq')
# emb_4_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-4/data/test_embeddings.pq')
# emb_14_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-14/data/test_embeddings.pq')
emb_15_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-15/data/test_embeddings.pq')
emb_17_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-17/data/test_embeddings.pq')
emb_18_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-18/data/test_embeddings.pq')
emb_21_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-21/data/test_embeddings.pq')
emb_22_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-22/data/test_embeddings.pq')

# emb_26_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-26/data/test_embeddings.pq')
# emb_27_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-27/data/test_embeddings.pq')
# emb_28_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-28/data/test_embeddings.pq')

In [19]:
from sklearn.preprocessing import normalize

def normalize_embeddings(embedding):
    return embedding / np.linalg.norm(embedding)

In [20]:
# emb_6_pd['embedding'] = emb_6_pd['embedding'].apply(normalize_embeddings)
# emb_4_pd['embedding'] = emb_4_pd['embedding'].apply(normalize_embeddings)
# emb_14_pd['embedding'] = emb_14_pd['embedding'].apply(normalize_embeddings)


emb_15_pd['embedding'] = emb_15_pd['embedding'].apply(normalize_embeddings)
emb_17_pd['embedding'] = emb_17_pd['embedding'].apply(normalize_embeddings)
emb_18_pd['embedding'] = emb_18_pd['embedding'].apply(normalize_embeddings)
emb_21_pd['embedding'] = emb_21_pd['embedding'].apply(normalize_embeddings)
emb_22_pd['embedding'] = emb_22_pd['embedding'].apply(normalize_embeddings)


# emb_26_pd['embedding'] = emb_26_pd['embedding'].apply(normalize_embeddings)
# emb_27_pd['embedding'] = emb_27_pd['embedding'].apply(normalize_embeddings)
# emb_28_pd['embedding'] = emb_28_pd['embedding'].apply(normalize_embeddings)

In [24]:
def merge_dfs(df1, df2, suffix1, suffix2):
    return df1.join(df2, how='inner', lsuffix=suffix1, rsuffix=suffix2)

In [25]:
# dataframes = [emb_6_pd, emb_4_pd, emb_14_pd] 
# dataframes = [emb_15_pd, emb_17_pd] 
# dataframes = [emb_15_pd, emb_17_pd, emb_18_pd]
# dataframes = [emb_15_pd, emb_17_pd, emb_18_pd, emb_21_pd, emb_22_pd, emb_26_pd, emb_27_pd, emb_28_pd]

dataframes = [emb_15_pd, emb_17_pd, emb_18_pd, emb_21_pd, emb_22_pd]
merged_df = dataframes[0]

suffixes = [f'_{i}' for i in range(len(dataframes))]

for i in range(1, len(dataframes)):
    merged_df = merged_df.join(dataframes[i], how='inner', lsuffix=suffixes[i-1], rsuffix=suffixes[i])

merged_df.rename(columns={'embedding': f'embedding_{len(dataframes)}'}, inplace=True)

In [1]:
# merged_df.head()

In [27]:
def average_embeddings(merged_df, weights=None):
    # Находим все столбцы, имена которых начинаются с 'embedding_'
    embedding_columns = [col for col in merged_df.columns if col.startswith('embedding_')]
    
    # Убедимся, что есть хотя бы один столбец для усреднения
    if not embedding_columns:
        raise ValueError("Нет столбцов с эмбеддингами для усреднения.")
    
    # Если веса не переданы, используем равные веса
    if weights is None:
        weights = np.ones(len(embedding_columns))
    elif len(weights) != len(embedding_columns):
        raise ValueError("Длина списка весов должна совпадать с количеством эмбеддингов.")

    # Применяем взвешенное усреднение по найденным столбцам
    merged_df['embedding'] = merged_df[embedding_columns].apply(
        lambda row: np.average(row, weights=weights), axis=1
    )

In [40]:
average_embeddings(merged_df, weights=[1.0, 0.9, 0.7, 0.9, 0.7])

In [2]:
# merged_df.head()

In [42]:
from sklearn.metrics import pairwise_distances_chunked

In [43]:
embeddings = merged_df['embedding'].tolist()
trackids = merged_df.index.tolist()

In [44]:
predictions = []
for chunk_result in tqdm(pairwise_distances_chunked(
        embeddings, metric='cosine', reduce_func=reduce_func, working_memory=100
)):
    for query_indx, query_nearest_items in chunk_result:
        predictions.append((trackids[query_indx], [trackids[nn_indx] for nn_indx in query_nearest_items]))

233it [03:03,  1.27it/s]


Тут указать директорию, в которой создаются ответы

In [45]:
save_test_predictions(
    predictions, output_dir='outputs_test', experiment_number='submission_final'
)